In [ ]:
import re
import json
from bson import json_util

date_format = '%Y-%m-%d %H:%M:%S'

def replace_matched(pattern, data):
    return re.sub(
            pattern,
            lambda match: match.group(2) if match.group(2) else '},',
            data,
            flags=re.MULTILINE
        )

def read_mongoextjson_file(filename):
    with open(filename, "r") as f:
        bsondata = f.read()
        pattern = r'(ObjectId|ISODate|Double|NumberInt)\s*\(\s*(\"?\S+\"?)\)|(^})(?=\n{)'
        return json.loads(f"[{replace_matched(pattern, bsondata)}]", object_hook=json_util.object_hook)

In [ ]:
def add_comma_between_objects(data):
    # Split the data into individual blocks and check if a comma is present after each block
    blocks = data.strip().split('\n}\n')
    formatted_data = ''
    for i, block in enumerate(blocks):
        if block.endswith(',') or i == len(blocks) - 1:
            formatted_data += block
        else:
            formatted_data += f"{block}\n}},"
        # Add a newline except for the last block
        if i != len(blocks) - 1:
            formatted_data += '\n'

    return formatted_data

In [ ]:
# activities
acols = ["event_time", "activity", "case", "task", "processing_time"]

# Setting default values for columns
# adf['case'] = adf['unique_id'].apply(lambda x: x['CaseID'] if x is not None else "None")
adf['case'] = adf['unique_id'].fillna('Missing').apply(lambda x: x.get('CaseID', 'Missing'))
adf['task'] = adf['activity_specifications'].fillna('Missing').apply(lambda x: x.get('task_name', 'Missing'))
# adf['task'] = adf['activity_specifications'].apply(lambda x: x['task_name'] if x is not None else "None")
# adf['activity'] = adf['activity_specifications'].apply(lambda x: x['activity_alias_name'] if x is not None else "None")
adf['activity'] = adf['activity_specifications'].fillna('Missing').apply(lambda x: x.get('activity_alias_name', 'Missing'))
adf['processing_time'] = adf['processing_time'].fillna(1, inplace=True) if 'processing_time' in adf.columns else 1

# Convert 'event_time' to datetime
date_format = '%Y-%m-%d %H:%M:%S'  # Change this to your desired date format
# adf['event_time'] = pd.to_datetime(adf['event_time']).dt.strftime(date_format)
adf['event_time'] = pd.to_datetime(adf['event_time'].fillna(pd.to_datetime('now'))).dt.strftime(date_format)